# AIA Week 2 Test - Where am I
An image classification task!
Using Transfer learning and CNN

## 問題定義

輸入資料是分為的15類不同場所的資料夾，最終任務是分辨 test set 的圖片是在哪裡。

## 引入套件

In [1]:
import pandas
import os
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras.preprocessing import image # 引入讀取圖片用的類別
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import Xception  #引入能夠架 Xception 的類別

Using TensorFlow backend.


## 資料預處理

In [2]:
# 確認資料位置
datadir = '//data/examples/may_the_4_be_with_u/where_am_i/' # 全部資料在 GPU server 的位置
traindir = datadir + 'train' # 圖片 training set 資料夾
testdir = datadir + 'testset'
print('training set from: %s\ntesting set from %s' % (traindir, testdir))

training set from: //data/examples/may_the_4_be_with_u/where_am_i/train
testing set from //data/examples/may_the_4_be_with_u/where_am_i/testset


### ImageDataGenerator
因為我們的資料及照片數量不夠多，因此透過[ImageDataGenerator](https://zhuanlan.zhihu.com/p/30197320)基於現有圖片來產生更多圖片。

可調的參數有:
- `rotation_range`:將圖片隨機選轉$[0,\theta]$內的角度。
- `rescale`：縮小圖片的倍數，通常會使用1/255，因為像素範圍是0~255。
- `width_shift_range/height_shift_rang`:水平、鉛直的平移，其參數可以是[0, 1]。
- `shear_range`:讓圖片傾斜的比例，讓所有X座標(或是Y座標)保持不變，對應的Y座標(或是X座標)按比例發生平移。
- `zoom_range`:此參數可以讓圖片的長或寬進行縮放，可以輸入一個數值或是list，若為數值，則兩軸按同樣比例縮放，若為list[width_zoom_range,height_zoom_range]，則表示寬高進行不同比例縮放。
- `horizontal_flip`:若為True，則隨機對圖片進行水平翻轉操作，通常不會進行垂直翻轉，因為圖片若垂直翻轉通常沒有意義。

In [3]:
datagen = ImageDataGenerator(rescale=1/255,
                             rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)

In [4]:
'''
利用 flow_from_directory 來讀入訓練資料
選擇拿取圖片的資料夾，還有決定圖片讀入的 size、訓練 batch_size
'''
train_gen = datagen.flow_from_directory(traindir, 
                                        target_size=(176,176),
                                        batch_size=32,
                                        class_mode="categorical")

Found 2985 images belonging to 15 classes.


## 建立模型
### 運用 Transfer Learning
利用 Imagenet pretrain 好的 Xception 方法作為 CNN 的底層架構，之後再接上 Fully connected layer 做分類。 
須先download weights, 會自動 download.

In [5]:
conv_base = Xception(weights="imagenet", include_top=False, input_shape=(176, 176, 3)) # 引用 Xception

In [6]:
model = models.Sequential() # 建一個空的 sequential 模型
model.add(conv_base) # 把前面設定好的 Xception 放進模型
model.add(layers.Flatten()) # 把 Xception 抽取出的圖片特徵攤平

model.add(layers.Dense(512, activation="relu", kernel_initializer='random_normal')) # 通過幾層 Dense 來做分類
model.add(layers.Dropout(0.3)) # 加 Dropout 避免 overfitting

model.add(layers.Dense(256, activation="relu", kernel_initializer='random_normal')) # 通過幾層 Dense 來做分類
model.add(layers.Dropout(0.3)) # 加 Dropout 避免 overfitting

model.add(layers.Dense(128, activation="relu", kernel_initializer='random_normal'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(128, activation="relu", kernel_initializer='random_normal'))
model.add(layers.Dropout(0.15))

model.add(layers.Dense(15, activation="softmax", kernel_initializer='random_normal'))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])

model.fit_generator(train_gen, steps_per_epoch=100, epochs=50) # 用前面建好的 generator 做訓練

Epoch 1/50
100/100 [==============================] - 32s 317ms/step - loss: 2.3707 - acc: 0.1994
Epoch 2/50
100/100 [==============================] - 26s 263ms/step - loss: 2.0544 - acc: 0.3102
Epoch 3/50
100/100 [==============================] - 26s 262ms/step - loss: 1.8350 - acc: 0.3929
Epoch 4/50
100/100 [==============================] - 27s 265ms/step - loss: 1.6310 - acc: 0.4657
Epoch 5/50
100/100 [==============================] - 26s 264ms/step - loss: 1.4581 - acc: 0.5019
Epoch 6/50
100/100 [==============================] - 26s 263ms/step - loss: 1.3459 - acc: 0.5445
Epoch 7/50
100/100 [==============================] - 27s 266ms/step - loss: 1.3948 - acc: 0.5345
Epoch 8/50
100/100 [==============================] - 27s 265ms/step - loss: 1.2525 - acc: 0.5777
Epoch 9/50
100/100 [==============================] - 27s 267ms/step - loss: 1.2067 - acc: 0.5859
Epoch 10/50
 54/100 [===============>..............] - ETA: 12s - loss: 1.1016 - acc: 0.6262

KeyboardInterrupt: 

In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 6, 6, 2048)        20861480  
_________________________________________________________________
flatten_5 (Flatten)          (None, 73728)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               37749248  
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               32896     
__________

## 資料預測
讀入測試資料並做預測，注意：測試資料就不用做擴增了，且記得把圖片的 size 跟 scale 變得跟訓練資料一樣。

In [31]:
id_list = [] # 用來存圖片檔名
class_list = [] # 用來存分類結果
submission_example = pandas.read_csv("./img-submission.csv", header=0)
submission_example = submission_example["id"]
for i in submission_example:
    i = i + ".jpg"
    file_name = os.path.join(testdir, i)
    img = image.load_img(file_name, target_size=(176, 176)) # 讀取圖片並調整 size
    img_array = image.img_to_array(img) # 將圖片轉成陣列
    img_array = img_array/255 # 跟訓練資料一樣做 rescale
    img_array = img_array.reshape((1,) + img_array.shape) # reshape 成模型能吃的形狀
    class_predict = model.predict_classes(img_array) # 對圖片做類別預測
    id_list.append(i[:-4]) # 把圖片檔名後的 .jpg 拿掉
    class_list.append(class_predict[0])

## Submission
上傳前需注意，使用 ImageDataGenerator 產生的 labelencoder 順序跟 mid_term_mapping.txt 文件順序不一樣，所以需要轉換。

In [14]:
labelmap = pd.read_csv(datadir + 'mid_term_mapping.txt', names=['place','index'])
labelmap

,place,index
0,CALsuburb,9
1,PARoffice,7
2,bedroom,12
3,coast,10
4,forest,4
5,highway,14
6,industrial,2
7,insidecity,3
8,kitchen,0
9,livingroom,5


In [15]:
sortlabel = labelmap.sort_values(by='index')
targetlist = sortlabel.place.as_matrix().tolist()
print(targetlist)

['kitchen', 'street', 'industrial', 'insidecity', 'forest', 'livingroom', 'opencountry', 'PARoffice', 'mountain', 'CALsuburb', 'coast', 'store', 'bedroom', 'tallbuilding', 'highway']


In [16]:
train_gen.class_indices # 觀察每個類別的labelencoder

{'CALsuburb': 0,
 'PARoffice': 1,
 'bedroom': 2,
 'coast': 3,
 'forest': 4,
 'highway': 5,
 'industrial': 6,
 'insidecity': 7,
 'kitchen': 8,
 'livingroom': 9,
 'mountain': 10,
 'opencountry': 11,
 'store': 12,
 'street': 13,
 'tallbuilding': 14}

## 將 predict 的 class 轉成 submission class

In [32]:
gen_class = train_gen.class_indices
inverdict = {k:v for v,k in gen_class.items()} # 將key與value對調
class_list_name = [inverdict[k] for k in class_list] # 透過對調的dict將class轉回placename
final_class = [targetlist.index(k) for k in class_list_name] # 轉換為正確的submission index

In [34]:
submission = pandas.DataFrame({"id": id_list, "class": final_class}) # 建 DataFrame 存放結果
submission.to_csv("submission.csv", index = False, columns = ["id", "class"]) # 輸出結果到 csv 檔

## My result:
![submission3](./submission3.png)

## Latest:
![submission6](./submission6.png)

## Ref:
- [AIA image classification: Where am I?](https://www.kaggle.com/c/aia-tc-image-cla-where-am-i/leaderboard)
- [classmate1](https://github.com/jerrywang8472/place_predict/blob/master/place_predict.ipynb)
- [classmate2](https://github.com/R7788380/AIA_ImageClassification/blob/master/Report.ipynb)
- [classmate3](https://github.com/stu12140513/test2/blob/master/aia.ipynb)
- [Sequential 顺序模型 API](https://keras.io/zh/models/sequential/)
- [无需数学背景，读懂 ResNet、Inception 和 Xception 三大变革性架构](https://www.jiqizhixin.com/articles/2017-08-19-4)